<a href="https://colab.research.google.com/github/c-c-c-c/yolo_sample/blob/master/yolo_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%bash
git clone https://github.com/thtrieu/darkflow.git


Cloning into 'darkflow'...


In [2]:

%mkdir darkflow/bin
%cd darkflow/bin


/content/darkflow/bin


In [3]:
# !wget https://pjreddie.com/media/files/yolo.weights

!wget https://pjreddie.com/media/files/yolov2.weights

--2020-06-29 04:05:33--  https://pjreddie.com/media/files/yolov2.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 203934260 (194M) [application/octet-stream]
Saving to: ‘yolov2.weights’

yolov2.weights      100%[===================>] 194.49M   275KB/s    in 10m 5s  

2020-06-29 04:15:40 (329 KB/s) - ‘yolov2.weights’ saved [203934260/203934260]



In [4]:
%ls ../
%pwd

bin/  darkflow/  flow*       LICENSE      README.md    setup.py
cfg/  demo.gif   labels.txt  preview.png  sample_img/  test/


'/content/darkflow/bin'

In [11]:
# !ls

bin  darkflow  flow	   LICENSE	README.md   setup.py
cfg  demo.gif  labels.txt  preview.png	sample_img  test


In [12]:
!python setup.py build_ext --inplace

Compiling darkflow/cython_utils/nms.pyx because it changed.
Compiling darkflow/cython_utils/cy_yolo2_findboxes.pyx because it changed.
Compiling darkflow/cython_utils/cy_yolo_findboxes.pyx because it changed.
[1/3] Cythonizing darkflow/cython_utils/cy_yolo2_findboxes.pyx
/usr/local/lib/python3.6/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/darkflow/darkflow/cython_utils/cy_yolo2_findboxes.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
[2/3] Cythonizing darkflow/cython_utils/cy_yolo_findboxes.pyx
/usr/local/lib/python3.6/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/darkflow/darkflow/cython_utils/cy_yolo_findboxes.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
[3/3] Cythonizing darkflow/cython_utils/nms.pyx
/usr/

In [5]:
%cd ../
!python setup.py build_ext --inplace

/content/darkflow
Processing /content/darkflow
  Created wheel for darkflow: filename=darkflow-1.0.0-cp36-cp36m-linux_x86_64.whl size=816620 sha256=b045a335e3afb869f3b602558312411ffa06de24db06bb6f42d5f4e5a7820a75
  Stored in directory: /tmp/pip-ephem-wheel-cache-klqwip9b/wheels/2f/3a/c5/e84e79d73d5a73aa1b5129a66a40947d9d77a32ebed501e431
Successfully built darkflow


In [6]:
%%bash
pip uninstall -y tensorflow
pip install tensorflow-gpu==1.15


Uninstalling tensorflow-2.2.0:
  Successfully uninstalled tensorflow-2.2.0
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=fee2db00ea98e5d78d69b24ac2977d9ef7c0d0bb4570e5856f8b92cc652440e1
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Found existing installation: tensorboard 2.2.2
    Uninstalling tensorboard-2.2.2:
      Successfully uninstalled tensorboard-2.2.2
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3


ERROR: tensorflow-probability 0.10.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.


In [7]:
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow
import cv2

In [8]:
# %cd darkflow/
%ls

bin/  darkflow/  flow*       LICENSE      README.md    setup.py
cfg/  demo.gif   labels.txt  preview.png  sample_img/  test/


In [9]:
%ls 

bin/  darkflow/  flow*       LICENSE      README.md    setup.py
cfg/  demo.gif   labels.txt  preview.png  sample_img/  test/


In [13]:
from darkflow.net.build import TFNet

options = {"model": "cfg/yolo.cfg", "load": "bin/yolov2.weights", "threshold": 0.1}
# options = {"model": "../library/darkflow/cfg/yolo.cfg", "load": "../library/darkflow/bin/yolo.weights", "threshold": 0.1}
 
tfnet = TFNet(options)
 
imgcv = cv2.imread("sample_img/sample_dog.jpg")
result = tfnet.return_predict(imgcv)
print(result)










Parsing cfg/yolo.cfg
Loading bin/yolov2.weights ...
Successfully identified 203934260 bytes
Finished in 0.021288633346557617s
Model has a coco model name, loading coco labels.

Building net ...

Source | Train? | Layer description                | Output size
-------+--------+----------------------------------+---------------



       |        | input                            | (?, 608, 608, 3)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 608, 608, 32)

 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 304, 304, 32)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 304, 304, 64)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 152, 152, 64)


/content/darkflow/darkflow/dark/darknet.py:54: UserWarning: ./cfg/yolov2.cfg not found, use cfg/yolo.cfg instead
  cfg_path, FLAGS.model))


 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 152, 152, 128)
 Load  |  Yep!  | conv 1x1p0_1  +bnorm  leaky      | (?, 152, 152, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 152, 152, 128)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 76, 76, 128)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 76, 76, 256)
 Load  |  Yep!  | conv 1x1p0_1  +bnorm  leaky      | (?, 76, 76, 128)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 76, 76, 256)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 38, 38, 256)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 38, 38, 512)
 Load  |  Yep!  | conv 1x1p0_1  +bnorm  leaky      | (?, 38, 38, 256)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 38, 38, 512)
 Load  |  Yep!  | conv 1x1p0_1  +bnorm  leaky      | (?, 38, 38, 256)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 38, 38, 512)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 19, 19, 512)
 Load  |  Yep! 